In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
kc1 = pd.read_csv(r'/content/kc1_csv.csv')
print('kc1 shape: ', kc1.shape)
kc1.head()

kc1 shape:  (2109, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,...,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,...,37,2,5,0,16.0,28.0,89.0,52.0,15.0,True
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,...,21,0,2,0,11.0,10.0,41.0,17.0,5.0,True


In [3]:
kc1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                2109 non-null   float64
 1   v(g)               2109 non-null   float64
 2   ev(g)              2109 non-null   float64
 3   iv(g)              2109 non-null   float64
 4   n                  2109 non-null   float64
 5   v                  2109 non-null   float64
 6   l                  2109 non-null   float64
 7   d                  2109 non-null   float64
 8   i                  2109 non-null   float64
 9   e                  2109 non-null   float64
 10  b                  2109 non-null   float64
 11  t                  2109 non-null   float64
 12  lOCode             2109 non-null   int64  
 13  lOComment          2109 non-null   int64  
 14  lOBlank            2109 non-null   int64  
 15  locCodeAndComment  2109 non-null   int64  
 16  uniq_Op            2109 

In [4]:
target = 'defects'
y = kc1[target]
x=kc1.drop(['defects'], axis=1)
x.shape

(2109, 21)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [6]:
logistic = LogisticRegression(C=1, penalty="l1", solver='liblinear', random_state=7).fit(x, y)
model = SelectFromModel(logistic, prefit=True)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [7]:
x_reduced = model.transform(x)
x_reduced.shape

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


(2109, 20)

In [8]:
y.shape

(2109,)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)

In [10]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2109, 21)
(2109,)
-------------------
(1476, 20)
(1476,)
(633, 20)
(633,)


In [11]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8499210110584519
Th confusion matrix of SVM:
  [[520  11]
 [ 84  18]]


In [12]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[520  11]
 [ 84  18]]
              precision    recall  f1-score   support

       False       0.86      0.98      0.92       531
        True       0.62      0.18      0.27       102

    accuracy                           0.85       633
   macro avg       0.74      0.58      0.60       633
weighted avg       0.82      0.85      0.81       633



In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.8467614533965245
The confusion matrix of KNN:
  [[520  11]
 [ 86  16]]
[[520  11]
 [ 86  16]]
              precision    recall  f1-score   support

       False       0.86      0.98      0.91       531
        True       0.59      0.16      0.25       102

    accuracy                           0.85       633
   macro avg       0.73      0.57      0.58       633
weighted avg       0.82      0.85      0.81       633



In [14]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.8309636650868878
[[488  43]
 [ 64  38]]
              precision    recall  f1-score   support

       False       0.88      0.92      0.90       531
        True       0.47      0.37      0.42       102

    accuracy                           0.83       633
   macro avg       0.68      0.65      0.66       633
weighted avg       0.82      0.83      0.82       633



In [15]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8238482384823849

In [16]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.88      0.92      0.90       531
        True       0.45      0.36      0.40       102

    accuracy                           0.83       633
   macro avg       0.67      0.64      0.65       633
weighted avg       0.81      0.83      0.82       633

[[486  45]
 [ 65  37]]
